# 뉴스 크롤링

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [69]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import datetime
from pytz import timezone
KST = timezone('Asia/Seoul')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)

## 기사 링크 크롤링

In [77]:
# 기사 링크 크롤링
# 카테고리당 100개 = 5페이지

six_url_list = []   # url 저장 리스트
entertainment_url_list = []
sports_url_list = []

category_list = []
categoty_name = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학"]

def getSixLink():    # 정치, 경제, 사회, 생활/문화, 세계, IT/과학
    a_list = []

    for category in range(3, 4):     # 6
        for page in range(1, 2):  # 1, 6
            url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
            browser.get(url)

            time.sleep(1)

            soup = BeautifulSoup(browser.page_source, "html.parser")
            a_list.extend(soup.select(".type06_headline dt+dt a"))
            a_list.extend(soup.select(".type06 dt+dt a"))

            category_list.extend([categoty_name[category] for _ in range(20)])
            print(f"{categoty_name[category]} {page} 페이지")

    for a in a_list:
        six_url_list.append(a["href"])


def getEntertainmentLink():   # 연예
    # today = str(datetime.datetime.now(KST))[:11]  # 서울 기준 시간
    a_list = []
    today = datetime.date.today()

    for page in range(1, 2):  # 1, 5
        url = f'https://entertain.naver.com/now#sid=106&date={today}&page={page}'
        browser.get(url)

        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        a_list.extend(soup.select(".news_lst li>a"))

        category_list.extend(["연예" for _ in range(25)])
        print(f"연예 {page} 페이지")

    for a in a_list:
        entertainment_url_list.append("https://entertain.naver.com" + a["href"])


def getSportsLink():    # 스포츠  (페이지마다 개수가 달라서 6페이지를 이동)
    # today = str(datetime.datetime.now(KST))[:11].replace('-', '')  # 서울 기준 시간
    a_list = []
    today = str(datetime.date.today()).replace('-', '')

    for page in range(1, 2):  # 1, 7
        url = f'https://sports.news.naver.com/general/news/index?isphoto=N&type=latest&date={today}&page={page}'
        browser.get(url)

        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")
        a_list.extend(soup.select(".news_list li>a"))

        print(f"스포츠 {page} 페이지")

    for i in range(len(a_list)):
        if i == 100:  # 100개 링크 추가했으면 멈추기
            break
        sports_url_list.append("https://sports.news.naver.com/news" + re.search('\?.+', a_list[i]["href"]).group())
        category_list.extend(["스포츠"])

getSixLink()
getEntertainmentLink()
# getSportsLink()


생활/문화 1 페이지
연예 1 페이지


In [78]:
print(len(six_url_list))
print(len(entertainment_url_list))
print(len(sports_url_list))

20
25
0


## 제목, 본문, 날짜 크롤링

In [67]:
# 수집한 태그 속에서 내용만 뽑아서 담는다.

def clean(article):   # 전처리
    article = re.sub('\w+ 기자','',article)
    article = re.sub('\w{2,4} 온라인 기자','',article)
    article = re.sub('\w{2,4}기자','',article)
    # article = re.sub('\S+@[a-z.]+','',article)

    article = re.sub('\n','',article)
    article = re.sub('\t','',article)
    article = re.sub('\u200b','',article)
    article = re.sub('\xa0','',article)
    article = re.sub('([a-zA-Z])','',article)
    article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
    article = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',article)
    article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)


    return article


In [79]:
# 수집한 링크의 제목, 본문, 날짜 크롤링

title_list = []     # 제목
content_list = []   # 본문
date_list = []      # 날짜

def getSixContent(url_list):   # 정치, 경제, 사회, 생활/문화, 세계, IT/과학
    title = []
    content = []
    date = []

    for url in url_list:
        browser.get(url)

        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select("#title_area span"))                 # 제목

        c = soup.find_all(attrs={"id" : "dic_area"})                  # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

        while c[0].find(attrs={"class" : "vod_player_wrap"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "vod_player_wrap"}).decompose()  # 본문 영상에 있는 글자 없애기

        content.extend(c)

        date.extend(soup.select("._ARTICLE_DATE_TIME"))               # 날짜

    for t in title:
        title_list.append(clean(t.text))

    for c in content:
        content_list.append(clean(c.text))

    for d in date:
        date_list.append(d["data-date-time"])


def getEntertainmentContent(url_list):    # 연예
    title = []
    content = []
    date = []

    for url in url_list:
        browser.get(url)

        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select(".end_tit"))                 # 제목

        c = soup.find_all(attrs={"id" : "articeBody"})                # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기
          if c[0].find(attrs={"class" : "caption"}):                  # 이미지 설명 없애기
            c[0].find(attrs={"class" : "caption"}).decompose()

        while c[0].find(attrs={"class" : "video_area"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "video_area"}).decompose()  # 본문 영상에 있는 글자 없애기

        while c[0].find(attrs={"name" : "iframe"}):
           c[0].find(attrs={"name" : "iframe"}).decompose()

        content.extend(c)

        date.extend(soup.select_one(".author em"))               # 날짜

    for t in title:
      title_list.append(clean(t.text))

    for c in content:
      content_list.append(clean(c.text))

    for d in date:
      date_list.append(d.text)


def getSportsContent(url_list):   # 스포츠
    title = []
    content = []
    date = []

    for url in url_list:
        print(url)
        browser.get(url)
        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select(".title"))                 # 제목

        c = soup.find_all(attrs={"class" : "news_end"})                # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

        while c[0].find(attrs={"class" : "image"}):
           c[0].find(attrs={"class" : "image"}).decompose()

        while c[0].find(attrs={"class" : "vod_area"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "vod_area"}).decompose()  # 본문 영상에 있는 글자 없애기

        content.extend(c)

        date.extend(soup.select_one(".info span"))               # 날짜

    for t in title:
      title_list.append(clean(t.text))

    for c in content:
      content_list.append(clean(c.text))

    for d in date:
      date_list.append(d.text)

getSixContent(six_url_list)
getEntertainmentContent(entertainment_url_list)
# getSportsContent(sports_url_list)


https://n.news.naver.com/mnews/article/001/0014305260?sid=103
https://n.news.naver.com/mnews/article/056/0011594468?sid=103
https://n.news.naver.com/mnews/article/001/0014305246?sid=103
https://n.news.naver.com/mnews/article/079/0003829552?sid=103
https://n.news.naver.com/mnews/article/032/0003258784?sid=103
https://n.news.naver.com/mnews/article/296/0000071064?sid=103
https://n.news.naver.com/mnews/article/018/0005610178?sid=103
https://n.news.naver.com/mnews/article/052/0001954797?sid=103
https://n.news.naver.com/mnews/article/022/0003870932?sid=103
https://n.news.naver.com/mnews/article/422/0000627197?sid=103
https://n.news.naver.com/mnews/article/088/0000843853?sid=103
https://n.news.naver.com/mnews/article/657/0000019914?sid=103
https://n.news.naver.com/mnews/article/056/0011594418?sid=103
https://n.news.naver.com/mnews/article/009/0005208710?sid=103
https://n.news.naver.com/mnews/article/032/0003258776?sid=103
https://n.news.naver.com/mnews/article/003/0012184424?sid=103
https://

In [80]:
print(len(title_list))
print(len(content_list))
print(len(date_list))

45
45
45


## 데이터 프레임 생성

In [ ]:
# 데이터 프레임으로 변환
all_url_list = six_url_list + entertainment_url_list + sports_url_list

article_df = pd.DataFrame({"카테고리" : category_list,
                           "날짜" : date_list,
                           "제목" : title_list,
                           "본문" : content_list,
                           "링크" : all_url_list})

article_df

# 전처리

**KoNLPy**
- 한국어 정보처리를 위한 파이썬 패키지
- https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class

In [ ]:
pip install konlpy

In [ ]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# CountVectorizer : 텍스트 빈도만을 카운트해서 벡터화
# TfidfVectorizer : 자주 나오는 단어에 높은 가중치, 모든 문서에서 자주 나오는 단어에 패널티

In [ ]:
okt = Okt()
nouns_list = []   # 명사 리스트

for content in article_df["본문"]:
  nouns_list.append(okt.nouns(content))     # 명사 추출, 리스트 반환

article_df["명사"] = nouns_list   # 데이터 프레임에 추가

In [ ]:
text = [" ".join(noun) for noun in article_df["명사"]]  # 명사 열을 하나의 리스트에 담는다.

tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1, 5))

# 군집화

In [ ]:
from sklearn.cluster import DBSCAN
# DBSCAN : 밀도 기반 클러스터링
# https://hoonzi-text.tistory.com/19